In [10]:
from abc import ABC, abstractmethod
import math
from datetime import datetime
import time
from scipy.stats import norm

class AbstractAsset(ABC):
    def __init__(self, name: str, price: float):
        self.name = name
        self.price = price
    def __repr__(self) -> str:
        return f'Stock: {self.name}, \nPrice: {self.price}'


class Stock:...

class Stock(AbstractAsset):

    __registry: dict[str, Stock] = dict()

    @classmethod
    def get_from_registry(cls, name: str) -> Stock:
        if (stock := cls.__registry.get(name)) is None: 
            raise KeyError(f"Stock {name} doesn't exist")
        return stock

    def __new__(cls, *args, **kwargs):
        instance = super().__new__(cls)
        name = kwargs.get('name')
        cls.__registry[name] = instance
        return instance
    
    
class Derivative(AbstractAsset):
    FORMAT_CODE = "%Y-%m-%d"
    def __init__(self, *args, expiration_date: str, underlying_name: str, **kwargs):
        super().__init__(*args, **kwargs)
        self.expiration_date = expiration_date
        self.__underlying: Stock = Stock.get_from_registry(underlying_name)
    
    @property
    def underlying(self) -> Stock:
        return self.__underlying

class Option(Derivative):
    def __init__(self, *args, strike_price: float, risk_free_rate: float, volatility: float, **kwargs):
        super().__init__(*args, **kwargs)
        self.strike_price = strike_price
        self.risk_free_rate = risk_free_rate
        self.volatility = volatility
        self.__payoff = None
            
    @abstractmethod
    def exercise(self):...


class EuropeanCallOption(Option):
    @property
    def payoff(self) -> float:
        T: float = datetime.strptime(self.expiration_date, self.FORMAT_CODE).timestamp() - time.time()
        T /= 60*60*24*365
        d1 = (math.log(self.underlying.price / self.strike_price) +\
             (self.risk_free_rate + self.volatility**2 / 2) * T) /\
             (self.volatility * math.sqrt(T))
        d2 = d1 - (self.volatility * math.sqrt(T))
        c = self.underlying.price * norm.cdf(d1) - self.strike_price * \
            math.exp(-self.risk_free_rate) * norm.cdf(d2)
        return c
    def exercise(self):
        return None


In [11]:
vtbr = Stock(name = 'VTBR', price = 70)

eur_call_option = EuropeanCallOption(
    name = 'Default Name', 
    strike_price = 70,
    risk_free_rate = 0.18,
    volatility = 0.3,
    expiration_date = '2025-12-31',
    price = 7,
    underlying_name = 'VTBR'
)

In [ ]:
sber: Stock = Stock(name = 'SBER', price = 300)
Stock.get_registry('SBER')

In [12]:
eur_call_option.payoff

np.float64(10.78882627148537)

In [ ]:
id(sber),\
id(Stock._Stock__registry['SBER'])